## Demographic inference of the divergence history of *Erythrura trichroa* and *Erythrura papuana*

This notebook performs a demographic inference of the divergence history of *Erythrura trichroa* and *Erythrura papuana* using `dadi2`. For now I run a limited number of replicates, to be boosted in the future. We start by importing the relevant libraries: 

In [1]:
import os
import numpy as np
import dadi
import nlopt
from dadi import Numerics
from scipy.optimize import minimize

Next we provide inputs for the analysis: our `.vcf` and population IDs and names: 

In [3]:
vcf = "/home/k14m234/erythrura_assembly/results/GCF_005870125.1/QC/erythrura.pruned.vcf.gz"
popfile = "/home/k14m234/erythrura/config/pixy_pop.txt"
pops = ['trichroa', 'papuana']

Directories for output: 

In [8]:
outdir = "/home/k14m234/erythrura/results/dadi"
os.makedirs(outdir, exist_ok=True)

Scaling constraints: 

In [19]:
mu = 2.3e-9            # per-site per-generation mutation rate
L  = 1010218420        # from awk '{sum += $3-$2} END{print sum}' erythrura_callable_sites.bed 
gen_time = 1.0         # years per generation (optional; set to 1 if unknown)

A folded SFS: 

In [14]:
dd = dadi.Misc.make_data_dict_vcf(vcf, popfile)
fs = dadi.Spectrum.from_data_dict(dd,
                                  pop_ids=['trichroa','papuana'],
                                  projections=[14, 18],   # or [16, 18]
                                  polarized=False)
print(fs.S(), fs.sample_sizes)

78626.47018553446 [14 18]


Sample sizes and grid sizes:  

In [15]:
ns = fs.sample_sizes
pts_l = [max(ns)+20, max(ns)+30, max(ns)+40]

Bootstrapped datasets:

In [16]:
# --- ADD 1: bootstrap spectra ONCE (outside loop) ---
Nboot = 100
chunk_size = int(5e6)
chunks = dadi.Misc.fragment_data_dict(dd, chunk_size)
boots = dadi.Misc.bootstraps_from_dd_chunks(chunks, Nboot, pops, ns)
boots = [b.fold() for b in boots]
for b in boots:
    b.mask = fs.mask

Define models:

In [17]:
# strict isolation
def SI(params, ns, pts):
    nu1, nu2, T = params
    xx = dadi.Numerics.default_grid(pts)
    phi = dadi.PhiManip.phi_1D(xx)
    phi = dadi.PhiManip.phi_1D_to_2D(xx, phi)
    phi = dadi.Integration.two_pops(phi, xx, T, nu1=nu1, nu2=nu2, m12=0, m21=0)
    return dadi.Spectrum.from_phi(phi, ns, (xx, xx))
si_ex = dadi.Numerics.make_extrap_log_func(SI)

# isolation-with-migration
def IM(params, ns, pts):
    nu1, nu2, T, m12, m21 = params
    xx = dadi.Numerics.default_grid(pts)
    phi = dadi.PhiManip.phi_1D(xx)
    phi = dadi.PhiManip.phi_1D_to_2D(xx, phi)
    phi = dadi.Integration.two_pops(phi, xx, T, nu1=nu1, nu2=nu2, m12=m12, m21=m21)
    fs_model = dadi.Spectrum.from_phi(phi, ns, (xx, xx))
    return fs_model
im_ex = dadi.Numerics.make_extrap_log_func(IM)

# secondary contact
def SC(params, ns, pts):
    nu1, nu2, T1, T2, m12, m21 = params
    xx = dadi.Numerics.default_grid(pts)
    phi = dadi.PhiManip.phi_1D(xx)
    phi = dadi.PhiManip.phi_1D_to_2D(xx, phi)

    # Phase 1: strict isolation
    phi = dadi.Integration.two_pops(phi, xx, T1, nu1=nu1, nu2=nu2, m12=0,   m21=0)

    # Phase 2: secondary contact
    phi = dadi.Integration.two_pops(phi, xx, T2, nu1=nu1, nu2=nu2, m12=m12, m21=m21)

    fs_model = dadi.Spectrum.from_phi(phi, ns, (xx, xx))
    return fs_model
sc_ex = dadi.Numerics.make_extrap_log_func(SC)

We now run replicate model fitting, starting with strict isolation (SI): 

In [20]:
# define starting param
SI_params = [1, 1, 0.1]
SI_param_names = ["nu1","nu2","T"]
SI_lower = [1e-3, 1e-3, 1e-4]
SI_upper = [10,   10,   5]

# checkoutput dir
os.makedirs("dadi", exist_ok=True)

# 20 replicates
for i in range(20):

    # random staring perturbation
    p0 = dadi.Misc.perturb_params(
        SI_params, fold=3,
        upper_bound=SI_upper,
        lower_bound=SI_lower
    )

    # generate model
    popt, ll_model = dadi.Inference.opt(
        p0, fs, si_ex, pts_l,
        lower_bound=SI_lower,
        upper_bound=SI_upper,
        algorithm=nlopt.LN_BOBYQA,
        log_opt=True, 
        maxeval=400,
        verbose=100
    )

    # get model fs
    model_fs = si_ex(popt, ns, pts_l)

    # theta + params
    theta0 = dadi.Inference.optimal_sfs_scaling(model_fs, fs)
    Nref   = theta0 / (4 * mu * L)
    nTri = popt[0] * Nref
    nPap = popt[1] * Nref
    t1  = popt[2] * 2 * Nref

    # write real params
    out1 = "\t".join(map(str, [i+1, ll_model, nTri, nPap, t1, theta0])) + "\n"
    with open("dadi/si_real.tsv", "a") as f:
        f.write(out1)

    # write model params
    out2 = "\t".join(map(str, [i+1, ll_model] + list(popt) + [theta0])) + "\n"
    with open("dadi/si_model.tsv", "a") as f:
        f.write(out2)

    for eps in [1e-2, 5e-3, 1e-3]:
        try:
            uncert = dadi.Godambe.GIM_uncert(si_ex, pts_l, boots, popt, fs, eps=eps)
        except np.linalg.LinAlgError:
            print(f"rep {i+1}: Godambe failed (singular); skipping CI")
            continue   # ← must be inside except block
    
        # Only runs if no exception
        if len(uncert) == len(popt) + 1:
            se = uncert[:-1]
        else:
            se = uncert
    
        ci_low  = np.array(popt) - 1.96 * np.array(se)
        ci_high = np.array(popt) + 1.96 * np.array(se)

    with open("dadi/si_ci.tsv", "a") as f:
        for name, est, se_i, lo, hi in zip(SI_param_names, popt, se, ci_low, ci_high):
            f.write("\t".join(map(str, [i+1, ll_model, name, est, se_i, lo, hi])) + "\n")

    print(i+1, ll_model, popt)

200     , -1636.15    , array([ 6.09835    ,  2.3682     ,  0.0701698  ])
1 -1636.0554441898744 [0.57255646 6.13446063 0.31263149]
300     , -1645.26    , array([ 7.69751    ,  2.33195    ,  0.0691813  ])
rep 2: Godambe failed (singular); skipping CI
2 -1636.0554442010161 [2.69899431 6.97786352 0.03754387]
400     , -1636.06    , array([ 6.08779    ,  2.34163    ,  0.0699959  ])
3 -1636.0554441926786 [0.74878992 0.83811642 0.38706408]
500     , -1638.49    , array([ 6.94606    ,  2.29276    ,  0.0702732  ])
rep 4: Godambe failed (singular); skipping CI
4 -1636.055444285374 [7.20865024 0.39603133 0.13578409]
600     , -1644.02    , array([ 7.82594    ,  2.2438     ,  0.0701135  ])
5 -1636.0554537465218 [7.09725114 0.13125081 0.11933351]
700     , -1888.62    , array([ 2.50047    ,  2.66979    ,  0.0639785  ])
6 -1636.0570172171938 [0.28859751 0.35123702 0.04144248]
800     , -2014.7     , array([ 3.45875    ,  1.38934    ,  0.0580568  ])
7 -1636.0554441918043 [0.21675893 0.51865859 0.07

900     , -1645.25    , array([ 7.61512    ,  2.23216    ,  0.0720979  ])
8 -1636.055449796348 [1.85242661 0.36422625 0.69744253]
1000    , -1664.45    , array([ 4.26838    ,  2.56263    ,  0.0691491  ])
9 -1636.0554445363234 [2.14571465 0.39170047 0.30814671]
1100    , -1637.6     , array([ 6.694      ,  2.28425    ,  0.0693551  ])
10 -1636.0554498204292 [1.3070313  0.27535018 0.03511345]
1200    , -1644.66    , array([ 7.41666    ,  2.23768    ,  0.067367   ])
11 -1636.0554443082906 [0.51027698 0.17439081 0.04439928]
1300    , -1639.5     , array([ 5.33645    ,  2.46736    ,  0.0700003  ])
12 -1636.055444142915 [4.4833333 4.6074795 0.0393724]
1400    , -1637.98    , array([ 6.09614    ,  2.22296    ,  0.0693269  ])
13 -1636.0554444848528 [0.95192977 6.23211765 0.07028477]
1500    , -1636.59    , array([ 5.8304     ,  2.39027    ,  0.0706179  ])
14 -1636.055444333301 [0.29793477 1.61804076 0.04860908]
1600    , -1636.09    , array([ 6.01006    ,  2.35133    ,  0.0700688  ])
15 -1636.0

4640.87259546238 78626.47018553446
1800    , -1916.51    , array([ 2.48633    ,  2.79409    ,  0.0566801  ])
18 -1636.0554444173977 [0.28101654 0.13849894 0.42535834]
1900    , -40985.2    , array([ 4.00012    ,  1.12276    ,  0.45964    ])
19 -1636.0554441974355 [4.00011756 1.12275511 0.03073795]
2000    , -3889.91    , array([ 0.844469   ,  0.621558   ,  0.0151936  ])
20 -1636.0554442513537 [0.54262108 0.72745711 0.07077948]


Next up is secondary contact (SC): 

In [21]:
# define starting param
SC_params = [1, 1, 0.2, 0.2, 0.01, 0.01]
SC_param_names = ["nu1","nu2","T1","T2","m12","m21"]
SC_lower = [1e-3, 1e-3, 1e-4, 1e-4, 1e-5, 1e-5]
SC_upper = [10,   10,   5,   5,   2,   2]

# checkoutput dir
os.makedirs("dadi", exist_ok=True)

# pick stepsize and write path for cis
eps = 1e-3
ci_path = "dadi/sc_ci.tsv"
if not os.path.exists(ci_path):
    with open(ci_path, "w") as f:
        f.write("rep\tll\tparam\test\tse\tci_low\tci_high\n")

# 20 replicates
for i in range(20):

    # random staring perturbation
    p0 = dadi.Misc.perturb_params(
        SC_params, fold=3,
        upper_bound=SC_upper,
        lower_bound=SC_lower
    )

    # generate model
    popt, ll_model = dadi.Inference.opt(
        p0, fs, sc_ex, pts_l,
        lower_bound=SC_lower,
        upper_bound=SC_upper,
        algorithm=nlopt.LN_BOBYQA,
        log_opt=True, 
        maxeval=400,
        verbose=100
    )

    # get model fs
    model_fs = sc_ex(popt, ns, pts_l)

    # theta + real params
    theta0 = dadi.Inference.optimal_sfs_scaling(model_fs, fs)
    Nref   = theta0 / (4 * mu * L)
    Nref=theta0/(4*mu*L)
    nTri=popt[0]*Nref
    nPap=popt[1]*Nref
    t1=popt[2]*2*Nref
    t2=popt[3]*2*Nref
    m12=popt[4]/(2*Nref)
    m21=popt[5]/(2*Nref)
    
    # write real params
    out1 = "\t".join(map(str, [i+1, ll_model, nTri, nPap, t1, t2, m12, m21, theta0])) + "\n"
    with open("dadi/sc_real.tsv", "a") as f:
        f.write(out1)

    # write model params
    out2 = "\t".join(map(str, [i+1, ll_model] + list(popt) + [theta0])) + "\n"
    with open("dadi/sc_model.tsv", "a") as f:
        f.write(out2)

    for eps in [1e-2, 5e-3, 1e-3]:
        try:
            uncert = dadi.Godambe.GIM_uncert(sc_ex, pts_l, boots, popt, fs, eps=eps)
        except np.linalg.LinAlgError:
            print(f"rep {i+1}: Godambe failed (singular); skipping CI")
            continue   # ← must be inside except block
    
        # Only runs if no exception
        if len(uncert) == len(popt) + 1:
            se = uncert[:-1]
        else:
            se = uncert
    
        ci_low  = np.array(popt) - 1.96 * np.array(se)
        ci_high = np.array(popt) + 1.96 * np.array(se)

    with open("dadi/sc_ci.tsv", "a") as f:
        for name, est, se_i, lo, hi in zip(SC_param_names, popt, se, ci_low, ci_high):
            f.write("\t".join(map(str, [i+1, ll_model, name, est, se_i, lo, hi])) + "\n")

    print(i+1, ll_model, popt)

2100    , -1947.97    , array([ 3.02154    ,  4.09261    ,  0.0527313  ,  0.0062535  ,  0.00391948 ,  0.0332504  ])
2200    , -1636.38    , array([ 6.16621    ,  2.31426    ,  0.0635964  ,  0.00647924 ,  0.00255553 ,  0.0145499  ])
2300    , -1636.14    , array([ 6.14259    ,  2.31302    ,  0.0647314  ,  0.00542784 ,  0.00276672 ,  0.0191243  ])
1 -1636.136680811123 [0.6315502  0.46260326 0.4875622  0.03881241 0.0044505  0.01524948]
2400    , -1636.84    , array([ 6.13426    ,  2.22076    ,  0.0204135  ,  0.0468871  ,  1.23475    ,  0.00450791 ])
2500    , -1598.43    , array([ 5.04587    ,  2.26495    ,  0.0180674  ,  0.0575326  ,  2          ,  0.00392807 ])
rep 2: Godambe failed (singular); skipping CI
2 -1598.3667032980738 [0.32716799 1.13799572 0.11260924 0.12970343 0.0614389  0.00720566]
2600    , -4240.52    , array([ 2.1767     ,  2.81262    ,  0.0479434  ,  0.0782807  ,  0.000114156,  1e-05      ])
2700    , -1636.81    , array([ 6.42599    ,  2.2848     ,  0.0341645  ,  0.035

Lastly, we'll run IM:

In [22]:
# define starting parameters
IM_params = [1, 1, 0.1, 0.01, 0.01]
IM_param_names = ["nu1","nu2","T","m12","m21"]
IM_lower = [1e-3, 1e-3, 1e-4, 1e-5, 1e-5]
IM_upper = [10,   10,   5,   2,   2]

# 20 replicates
for i in range(20):

    # random staring perturbation
    p0 = dadi.Misc.perturb_params(
        IM_params, fold=3,
        upper_bound=IM_upper,
        lower_bound=IM_lower
    )

    # generate model
    popt, ll_model = dadi.Inference.opt(
        p0, fs, im_ex, pts_l,
        lower_bound=IM_lower,
        upper_bound=IM_upper,
        algorithm=nlopt.LN_BOBYQA,
        log_opt=True, 
        maxeval=400,
        verbose=100
    )

    # get model fs
    model_fs = im_ex(popt, ns, pts_l)

    # theta + real params
    theta0 = dadi.Inference.optimal_sfs_scaling(model_fs, fs)
    Nref   = theta0 / (4 * mu * L)
    Nref=theta0/(4*mu*L)
    nTri=popt[0]*Nref
    nPap=popt[1]*Nref
    t1=popt[2]*2*Nref
    m12=popt[3]/(2*Nref)
    m21=popt[4]/(2*Nref)
    
    # write real params
    out1 = "\t".join(map(str, [i+1, ll_model, nTri, nPap, t1, m12, m21, theta0])) + "\n"
    with open("dadi/im_real.tsv", "a") as f:
        f.write(out1)

    # write model params
    out2 = "\t".join(map(str, [i+1, ll_model] + list(popt) + [theta0])) + "\n"
    with open("dadi/im_model.tsv", "a") as f:
        f.write(out2)

    for eps in [1e-2, 5e-3, 1e-3]:
        try:
            uncert = dadi.Godambe.GIM_uncert(im_ex, pts_l, boots, popt, fs, eps=eps)
        except np.linalg.LinAlgError:
            print(f"rep {i+1}: Godambe failed (singular); skipping CI")
            continue   # ← must be inside except block
    
        # Only runs if no exception
        if len(uncert) == len(popt) + 1:
            se = uncert[:-1]
        else:
            se = uncert
    
        ci_low  = np.array(popt) - 1.96 * np.array(se)
        ci_high = np.array(popt) + 1.96 * np.array(se)

    with open("dadi/im_ci.tsv", "a") as f:
        for name, est, se_i, lo, hi in zip(IM_param_names, popt, se, ci_low, ci_high):
            f.write("\t".join(map(str, [i+1, ll_model, name, est, se_i, lo, hi])) + "\n")
    
    print(i+1, ll_model, popt)

7300    , -1635.98    , array([ 5.98783    ,  2.32448    ,  0.0699047  ,  0.00517026 ,  0.00399877 ])
7400    , -1633.8     , array([ 5.9012     ,  2.362      ,  0.0706819  ,  0.0990816  ,  0.00708979 ])
1 -1633.4370938682964 [0.7481295  0.3177757  0.01655758 0.0332887  0.00902265]
7500    , -1812.63    , array([ 10         ,  2.33524    ,  0.0891591  ,  0.00661523 ,  0.0121236  ])
7600    , -1635.62    , array([ 6.05365    ,  2.34268    ,  0.0699559  ,  0.00948752 ,  0.014313   ])
7700    , -1633.94    , array([ 6.28764    ,  2.29785    ,  0.0702075  ,  0.0262318  ,  0.106937   ])
2 -1633.7172059282755 [1.02276899 2.26492847 0.03220926 0.00531132 0.01692385]
7800    , -28939.7    , array([ 6.76594    ,  0.780062   ,  0.273116   ,  0.00320036 ,  0.0335931  ])
7900    , -1635.19    , array([ 5.95562    ,  2.32785    ,  0.0699596  ,  0.00985742 ,  0.0447844  ])
8000    , -1634.49    , array([ 6.23095    ,  2.2613     ,  0.0703654  ,  0.0253318  ,  0.0928973  ])
3 -1633.6037747384933 [2.0

Write parameters with CIs: